In [1]:
# -*- coding: utf-8 -*-
"""NLP HW1 .ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1T4PnsiYWNGlhunO0vTqcDOFYoZGc8153
"""

# question 2.2 task 1.
from collections import Counter, defaultdict

corpus = ["new", "newer", "lowest", "widest"]

# initialize with characters + end-of-word
def init_vocab(words):
    return {tuple(list(w) + ["_"]): 1 for w in words}

def get_pair_counts(vocab):
    counts = Counter()
    for tokens, freq in vocab.items():
        for i in range(len(tokens)-1):
            counts[(tokens[i], tokens[i+1])] += freq
    return counts

def merge_pair(pair, vocab):
    new_vocab = {}
    a, b = pair
    for tokens, freq in vocab.items():
        new = []
        i = 0
        while i < len(tokens):
            if i < len(tokens)-1 and tokens[i] == a and tokens[i+1] == b:
                new.append(a+b)
                i += 2
            else:
                new.append(tokens[i])
                i += 1
        new_vocab[tuple(new)] = freq
    return new_vocab

vocab = init_vocab(corpus)
symbols = set(ch for w in corpus for ch in w) | {"_"}

for step in range(1, 8):
    pair_counts = get_pair_counts(vocab)
    if not pair_counts: break
    top_pair = pair_counts.most_common(1)[0][0]
    vocab = merge_pair(top_pair, vocab)
    symbols.add("".join(top_pair))
    print(f"Step {step}: top pair = {top_pair}, vocab size = {len(symbols)}")

from collections import Counter

# -----------------------------
# 2.3 — BPE on a short paragraph
# -----------------------------

PARAGRAPH = (
    "Natural language processing helps computers understand human language. "
    "Language models learn patterns from large text collections. "
    "Tokenization breaks words into smaller meaningful units. "
    "Subword methods improve handling of unknown words. "
    "Researchers continuously improve language technologies."
)

EOW = "_"          # end-of-word marker (assignment requirement)
NUM_MERGES = 30    # learn at least 30 merges


# ---------- Helpers ----------
def words_from_paragraph(text: str):
    # Basic word extraction: keep letters only, lowercase
    cleaned = []
    cur = []
    for ch in text.lower():
        if ch.isalpha():
            cur.append(ch)
        else:
            if cur:
                cleaned.append("".join(cur))
                cur = []
    if cur:
        cleaned.append("".join(cur))
    return cleaned

def init_corpus(words):
    # Represent each word as a list of characters + end-of-word marker
    return [list(w) + [EOW] for w in words]

def get_pair_counts(corpus):
    counts = Counter()
    for tokens in corpus:
        for i in range(len(tokens) - 1):
            counts[(tokens[i], tokens[i+1])] += 1
    return counts

def merge_pair_in_tokens(tokens, pair):
    a, b = pair
    merged = []
    i = 0
    while i < len(tokens):
        if i < len(tokens) - 1 and tokens[i] == a and tokens[i+1] == b:
            merged.append(a + b)
            i += 2
        else:
            merged.append(tokens[i])
            i += 1
    return merged

def apply_merge_to_corpus(corpus, pair):
    return [merge_pair_in_tokens(tokens, pair) for tokens in corpus]

def learn_bpe(words, num_merges=30):
    corpus = init_corpus(words)
    merges = []              # list of pairs in the order learned
    merge_freqs = []         # frequency of the chosen pair at that step

    # initial vocabulary = all unique symbols (characters + EOW)
    vocab = set(sym for w in corpus for sym in w)

    for step in range(1, num_merges + 1):
        pair_counts = get_pair_counts(corpus)
        if not pair_counts:
            break

        top_pair, top_count = pair_counts.most_common(1)[0]

        # record
        merges.append(top_pair)
        merge_freqs.append((top_pair, top_count))

        # merge everywhere in corpus
        corpus = apply_merge_to_corpus(corpus, top_pair)

        # add merged symbol to vocabulary
        vocab.add(top_pair[0] + top_pair[1])

        print(f"Step {step:02d}: top pair = {top_pair}  |  count = {top_count:3d}  |  vocab size = {len(vocab)}")

    return merges, merge_freqs, vocab

def bpe_segment_word(word, merges):
    # segmenter: apply merges greedily, in learned order
    tokens = list(word.lower()) + [EOW]
    for pair in merges:
        tokens = merge_pair_in_tokens(tokens, pair)
    return tokens


# ---------- Run BPE ----------
words = words_from_paragraph(PARAGRAPH)
merges, merge_freqs, vocab = learn_bpe(words, NUM_MERGES)

print("\n--- Five most frequent merges (by frequency at time learned) ---")
top5_merges = sorted(merge_freqs, key=lambda x: x[1], reverse=True)[:5]
for pair, cnt in top5_merges:
    print(f"{pair}  ->  {pair[0] + pair[1]}   (count={cnt})")

print("\n--- Five longest subword tokens in final vocabulary ---")
# longest tokens by string length (excluding single chars if you want, but we keep all)
longest5 = sorted(vocab, key=lambda s: (len(s), s), reverse=True)[:5]
for tok in longest5:
    print(tok)

# ---------- Segment 5 words from the paragraph ----------
# Pick 5 words present in the paragraph; include one rare and one derived/inflected form.
# (Example choices: "tokenization" (derived), "collections" (rarer), plus a few others)
targets = ["tokenization", "collections", "processing", "unknown", "technologies"]

print("\n--- Segmentations (tokens include end-of-word _ where applicable) ---")
for w in targets:
    seg = bpe_segment_word(w, merges)
    print(f"{w:14s} -> {' '.join(seg)}")

#Question 5. 1.
# Naive space-based tokenization (Telugu or any language)
# Splits ONLY on whitespace, so punctuation stays attached.

text = "నేను ఈ సినిమా చాలా బాగుంది! ఇది ఈ సంవత్సరం చూసిన ఉత్తమ చిత్రాల్లో ఒకటి. చూడండి, మీరు ఖచ్చితంగా ఇష్టపడతారు."

tokens = text.split()   # naive whitespace split
print(tokens)

!pip install indic-nlp-library

#question 5.2. Tool comparision

from indicnlp.tokenize import indic_tokenize

text = "నేను ఈ సినిమా చాలా బాగుంది! ఇది ఈ సంవత్సరం చూసిన ఉత్తమ చిత్రాల్లో ఒకటి. చూడండి, మీరు ఖచ్చితంగా ఇష్టపడతారు."

tool_tokens = indic_tokenize.trivial_tokenize(text, lang="te")
print(tool_tokens)

Step 1: top pair = ('n', 'e'), vocab size = 12
Step 2: top pair = ('ne', 'w'), vocab size = 13
Step 3: top pair = ('e', 's'), vocab size = 14
Step 4: top pair = ('es', 't'), vocab size = 15
Step 5: top pair = ('est', '_'), vocab size = 16
Step 6: top pair = ('new', '_'), vocab size = 17
Step 7: top pair = ('new', 'e'), vocab size = 18
Step 01: top pair = ('s', '_')  |  count =  12  |  vocab size = 26
Step 02: top pair = ('a', 'n')  |  count =   8  |  vocab size = 27
Step 03: top pair = ('e', '_')  |  count =   7  |  vocab size = 28
Step 04: top pair = ('g', 'e_')  |  count =   5  |  vocab size = 29
Step 05: top pair = ('i', 'n')  |  count =   5  |  vocab size = 30
Step 06: top pair = ('e', 'r')  |  count =   5  |  vocab size = 31
Step 07: top pair = ('l', 'an')  |  count =   4  |  vocab size = 32
Step 08: top pair = ('lan', 'g')  |  count =   4  |  vocab size = 33
Step 09: top pair = ('lang', 'u')  |  count =   4  |  vocab size = 34
Step 10: top pair = ('langu', 'a')  |  count =   4  |